In [1]:
%load_ext autoreload
%autoreload 2

In [9]:
# !pip install spacy
# !python -m spacy download en_core_web_sm

In [5]:
import spacy

In [8]:
nlp = spacy.load("en_core_web_sm")

In [21]:
sentences = [
    "The food we had yesterday was delicious",
    "My time in Italy was very enjoyable",
    "I found the meal to be tasty",
    "The Internet was so slow",
    "Our experience was suboptimal",
]

In [22]:
# split sentences so that we have target aspects and their sentiments descriptions
for sentence in sentences:
    doc = nlp(sentence)
    for token in doc:
        print(token.text, token.dep_, token.head.text, token.head.pos_, token.pos_, [child for child in token.children])

The det food NOUN DET []
food nsubj was AUX NOUN [The, had]
we nsubj had VERB PRON []
had relcl food NOUN VERB [we, yesterday]
yesterday npadvmod had VERB NOUN []
was ROOT was AUX AUX [food, delicious]
delicious acomp was AUX ADJ []
My poss time NOUN PRON []
time nsubj was AUX NOUN [My, in]
in prep time NOUN ADP [Italy]
Italy pobj in ADP PROPN []
was ROOT was AUX AUX [time, enjoyable]
very advmod enjoyable ADJ ADV []
enjoyable acomp was AUX ADJ [very]
I nsubj found VERB PRON []
found ROOT found VERB VERB [I, be]
the det meal NOUN DET []
meal nsubj be AUX NOUN [the]
to aux be AUX PART []
be ccomp found VERB AUX [meal, to, tasty]
tasty acomp be AUX ADJ []
The det Internet NOUN DET []
Internet nsubj was AUX NOUN [The]
was ROOT was AUX AUX [Internet, slow]
so advmod slow ADJ ADV []
slow acomp was AUX ADJ [so]
Our poss experience NOUN PRON []
experience nsubj was AUX NOUN [Our]
was ROOT was AUX AUX [experience, suboptimal]
suboptimal acomp was AUX ADJ []


In [24]:
# check sentiment descriptions in sentences [adj in sentences], adding intensifiers -> it picks up adverbs (checks for child tokens for each adjective
# and picks up adverbs like very, quite, so ,etc

aspects = []
for sentence in sentences:
    doc = nlp(sentence)
    descriptive_term = ''
    target = ''
    desc_term_list = []
    for token in doc:
        if token.dep_ == 'nsubj' and token.pos_ == 'NOUN':
            target = token.text
        if token.pos_ == "ADJ":
            prepend = ''
            for child in token.children:
                if child.pos_ != 'ADV':
                    continue
                prepend += child.text + ' '
            descriptive_term = prepend + token.text    
            desc_term_list.append(descriptive_term)
    # print(sentence)
    # print(descriptive_term)
    # print(desc_term_list)
    aspects.append({"aspect": target, "description": descriptive_term})
print(aspects)

[{'aspect': 'food', 'description': 'delicious'}, {'aspect': 'time', 'description': 'very enjoyable'}, {'aspect': 'meal', 'description': 'tasty'}, {'aspect': 'Internet', 'description': 'so slow'}, {'aspect': 'experience', 'description': 'suboptimal'}]


Using TextBlob sentiment analysis

In [26]:
# !pip install TextBlob

In [27]:
from textblob import TextBlob

In [28]:
for aspect in aspects:
    aspect["sentiment"] = TextBlob(aspect["description"]).sentiment
print(aspects)

[{'aspect': 'food', 'description': 'delicious', 'sentiment': Sentiment(polarity=1.0, subjectivity=1.0)}, {'aspect': 'time', 'description': 'very enjoyable', 'sentiment': Sentiment(polarity=0.65, subjectivity=0.78)}, {'aspect': 'meal', 'description': 'tasty', 'sentiment': Sentiment(polarity=0.0, subjectivity=0.0)}, {'aspect': 'Internet', 'description': 'so slow', 'sentiment': Sentiment(polarity=-0.30000000000000004, subjectivity=0.39999999999999997)}, {'aspect': 'experience', 'description': 'suboptimal', 'sentiment': Sentiment(polarity=0.0, subjectivity=0.0)}]


Training NaiveBayes Classifier for TextBlob

In [32]:
# !python -m textblob.download_corpora

In [33]:
from textblob.classifiers import NaiveBayesClassifier

In [34]:
train = [("Slow internet", "negative"),
        ("delicious food", "positive"),
        ("suboptimal experience", "negative"),
        ("very enjoyable time", "positive"),
        ]
cl = NaiveBayesClassifier(train)

In [38]:
blob = TextBlob("The food was delicious. the experience was suboptimal", classifier=cl)

In [40]:
for s in blob.sentences:
    print(s, "-", s.classify())

The food was delicious. - positive
the experience was suboptimal - negative
